In [1]:
__author__ = 'Chien-Hsiu Lee <chien-hsiu.lee@noirlab.edu>, Thomas Matheson <tom.matheson@noirlab.edu>'
__version__ = '20240601' # yyyymmdd
__datasets__ = ['']
__keywords__ = ['ANTARES', 'variable']

# Exploring Elastic Search Database to Find R Corona Borealis Stars

*Chien-Hsiu Lee, Thomas Matheson, & the ANTARES Team*

### Table of contents
* [Goals & notebook summary](#goals)
* [Disclaimer & attribution](#attribution)
* [Imports & setup](#import)
* [Querying ANTARES alert database ](#query)
* [Inspecting light curves](#lightcurve)
* [Resources and references](#resources)

<a class="anchor" id="goals"></a>
# Goals
This notebook is an example of how to explore the ANTARES alert database for variable stars. Here we use the infrared color selection of candidate R Coronae Borealise stars, and search the ZTF time-series photometry to see if there are unknown R CrBs revealing themselves by a significant, irregular dimming (up to 8 mag).

# Summary
We first obtain the candidate R CrBs from WISE color, selected by Tisserand et al. (2012). We then use the ANTARES search API to find time-series photometry of each candidate, and looked for R CrBs candidates that show more than 2 magnitude variability in either g- or r-band. In the end, we use ZTF18abhjrcf as a showcase.

<img src="RCB.jpeg">

<a class="anchor" id="attribution"></a>
# Disclaimer & attribution

Disclaimers
-----------
Note that using the Astro Data Lab constitutes your agreement with our minimal [Disclaimers](https://datalab.noirlab.edu/disclaimers.php).

Acknowledgments
---------------
If you use **Astro Data Lab** in your published research, please include the text in your paper's Acknowledgments section:

_This research uses services or data provided by the Astro Data Lab, which is part of the Community Science and Data Center (CSDC) Program of NSF NOIRLab. NOIRLab is operated by the Association of Universities for Research in Astronomy (AURA), Inc. under a cooperative agreement with the U.S. National Science Foundation._

If you use **SPARCL jointly with the Astro Data Lab platform** (via JupyterLab, command-line, or web interface) in your published research, please include this text below in your paper's Acknowledgments section:

_This research uses services or data provided by the SPectra Analysis and Retrievable Catalog Lab (SPARCL) and the Astro Data Lab, which are both part of the Community Science and Data Center (CSDC) Program of NSF NOIRLab. NOIRLab is operated by the Association of Universities for Research in Astronomy (AURA), Inc. under a cooperative agreement with the U.S. National Science Foundation._

In either case **please cite the following papers**:

* Data Lab concept paper: Fitzpatrick et al., "The NOAO Data Laboratory: a conceptual overview", SPIE, 9149, 2014, https://doi.org/10.1117/12.2057445

* Astro Data Lab overview: Nikutta et al., "Data Lab - A Community Science Platform", Astronomy and Computing, 33, 2020, https://doi.org/10.1016/j.ascom.2020.100411

If you are referring to the Data Lab JupyterLab / Jupyter Notebooks, cite:

* Juneau et al., "Jupyter-Enabled Astrophysical Analysis Using Data-Proximate Computing Platforms", CiSE, 23, 15, 2021, https://doi.org/10.1109/MCSE.2021.3057097

If publishing in a AAS journal, also add the keyword: `\facility{Astro Data Lab}`

And if you are using SPARCL, please also add `\software{SPARCL}` and cite:

* Juneau et al., "SPARCL: SPectra Analysis and Retrievable Catalog Lab", Conference Proceedings for ADASS XXXIII, 2024
https://doi.org/10.48550/arXiv.2401.05576

The NOIRLab Library maintains [lists of proper acknowledgments](https://noirlab.edu/science/about/scientific-acknowledgments) to use when publishing papers using the Lab's facilities, data, or services.

<a class="anchor" id="import"></a>
# Imports and setup

In [2]:
from antares_client.search import search
import matplotlib.pyplot as plt
import pandas as pd

<a class="anchor" id="import"></a>
# Read in relevant tables

We use the candidate list from Tisserand (2012), dropping candidates below Dec<-30 (in the ZTF field). Here we read in the list of candidate, specifically their ra and dec.

In [3]:
catalog=pd.read_csv('WISE_RCrB.dat')
ra=catalog['ra']
dec=catalog['dec']

<a class="anchor" id="query"></a>
# Querying ANTARES alert database 

This cell shows how to call elastic search with ANTARES API (the antares_client we imported earlier). It can search on ZTF object id, RA, Dec, or other properties. For our purpose, we search for variabilities larger than 2 mag in either g- or r-band. For illustration purpose, we only search variability in three of the candidate (id = 228-230). 

In ANTARES we aggregrate alerts on the same point of sky to form a locus object (which we shall call it "locus" hereafter). If there is a match at the given position, the search will return the corresponding locus. An astropy.TimeSeries of all Alert Properties on this locus can be obtained using locus.timeseries. We can convert it to Panda dataframe using locus.timeseries.to_pandas() for futher analysis. Details of the locus object can be found in the ANTARES documentation at: https://noao.gitlab.io/antares/filter-documentation/devkit/locus.html

In [4]:
import astropy
import warnings

for i in [228, 229, 230]:  
    print("Iteration: ",i, "ra",ra[i],"dec",dec[i])
    query = {
        "query": {
            "bool": {
                "must": [
                    {
                        "range": {
                            "ra": {
                                "gte": ra[i]-1./3600.,
                                "lte": ra[i]+1./3600.,
                            }
                        }
                    },
                    {
                        "range": {
                            "dec": {
                                "gte": dec[i]-1./3600.,
                                "lte": dec[i]+1./3600.
                            }
                        }
                    }
                 ]
            }
        }
    }
    for locus in search(query):
        with warnings.catch_warnings():
            # The cast of locus.timeseries: astropy.table.Table to a pandas
            # dataframe results in the conversion of some integer-valued
            # columns to floating point represntation. This can result in a
            # number of noisy warning so we will catch & ignore them for the
            # next couple of lines.
            warnings.simplefilter("ignore", astropy.table.TableReplaceWarning)
            df = locus.timeseries.to_pandas()
        df = df[['ant_mjd', 'ztf_fid', 'ztf_magpsf', 'ztf_sigmapsf']]
        dn = df.dropna()
        if max(dn['ztf_magpsf'][dn['ztf_fid']==1]) - min(dn['ztf_magpsf'][dn['ztf_fid']==1]) > 2:
            print("Got a hit on locus_id: ",locus.locus_id," in g-band")
        if max(dn['ztf_magpsf'][dn['ztf_fid']==2]) - min(dn['ztf_magpsf'][dn['ztf_fid']==2]) > 2:
            print("Got a hit on locus_id: ",locus.locus_id," in r-band")

Iteration:  228 ra 275.832794 dec -14.109205
Iteration:  229 ra 283.318207 dec -27.231319
Got a hit on locus_id:  ANT2020emqbg  in g-band
Got a hit on locus_id:  ANT2020emqbg  in r-band
Iteration:  230 ra 280.692779 dec -12.904075
Got a hit on locus_id:  ANT2018c7igm  in g-band
Got a hit on locus_id:  ANT2018c7igm  in r-band


<a class="anchor" id="lightcurve"></a>
# Inspecting light curves

Looks like we got a hit. Let's have a look at the last one (locus_id ANT2018c7igm). Since we have the time-series information in hand from previous cell, we can plot the light curve using the following cell:

In [5]:
plt.ylim(max(dn['ztf_magpsf'])+0.1*(max(dn['ztf_magpsf'])-min(dn['ztf_magpsf'])),min(dn['ztf_magpsf'])-0.1*(max(dn['ztf_magpsf'])-min(dn['ztf_magpsf'])))
plt.scatter(dn['ant_mjd'][dn['ztf_fid']==2], dn['ztf_magpsf'][dn['ztf_fid']==2], c='red', alpha=0.5)
plt.scatter(dn['ant_mjd'][dn['ztf_fid']==1], dn['ztf_magpsf'][dn['ztf_fid']==1], c='green', alpha=0.5)
plt.title('Light curve of Locus_id=ANT2018c7igm')
plt.xlabel('Time [Julian date]')
plt.ylabel('Magnitude in g- and r-passband')
plt.show()

# Concluding remarks

Locus_id ANT2018c7igm (=ZTF18abhjrcf) shows a rapid change of more than 2 magnitudes in r-band, and consistent brightness change in the g-band as well (thought not as much as r-band). This is similar to the sudden change in brightness seen in R Coronae Borealis. We subsequently obtained spectra of this source, and confirmed its RCB nature.

# Resources and references
Further reading:

AAVSO introduction on R Coronae Borealis stars: https://www.aavso.org/vsots_rcrb

Tisserand (2012) "Tracking down R Coronae Borealis stars from their mid-infrared WISE colours". A&A, 539, 51: https://ui.adsabs.harvard.edu/abs/2012A&A...539A..51T

Tisserand et al. (2013) "The ongoing pursuit of R Coronae Borealis stars: the ASAS-3 survey strikes again". A&A, 551, 22: https://ui.adsabs.harvard.edu/abs/2013A&A...551A..77T